In [1]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

from src.data import cleandata

In [2]:
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de COMUNAS OBTENIDAS DEL COT

In [3]:
# Analizo contra los datos extraidos en COT

df_comunas = pd.read_feather("../data/interim/subdere/Provincias_feather")
comunas = df_comunas['Nombre Comuna'].unique()
len(comunas)

346

## DATA CENSO 2017 : FUENTE INE

In [4]:
df_censo = pd.read_excel("../data/external/censo/1_1_POBLACION.xls", sheet_name = "Comuna")
df_censo.head()

WARNING *** file size (9789312) not 512 + multiple of sector size (512)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,1.1 Población total por sexo y área urbana-rur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,ORDEN,NOMBRE REGIÓN,CÓDIGO REGIÓN,NOMBRE PROVINCIA,CÓDIGO PROVINCIA,NOMBRE COMUNA,CÓDIGO COMUNA,EDAD,TOTAL POBLACIÓN EFECTIVAMENTE CENSADA,HOMBRES,MUJERES,TOTAL ÁREA URBANA,HOMBRES ÁREA URBANA,MUJERES ÁREA URBANA,TOTAL ÁREA RURAL,HOMBRES ÁREA RURAL,MUJERES ÁREA RURAL
2,NaN,0,PAÍS,País,PAÍS,País,PAÍS,País,Total País,17574003,8601989,8972014,15424263,7466848,7957415,2149740,1135141,1014599
3,NaN,1,PAÍS,País,PAÍS,País,PAÍS,País,0,214682,109390,105292,191763,97904,93859,22919,11486,11433
4,NaN,2,PAÍS,País,PAÍS,País,PAÍS,País,1,236356,120289,116067,210190,106888,103302,26166,13401,12765


In [5]:
df_censo.drop(['Unnamed: 0'], axis='columns', inplace=True)
df_censo.drop(0, axis='rows', inplace=True)

In [6]:
old_columns = []
for col in range(1,18):
    old_columns.append(f"Unnamed: {col}")

new_columns = []
for col in old_columns:
    new_columns.append(df_censo[col][1])

columnas = dict(zip(old_columns, new_columns))

df_censo.rename(columns=columnas, inplace=True)

In [7]:
cols = df_censo.select_dtypes(include = ["object"]).columns
df_censo[cols] = df_censo[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 17/17 [00:01<00:00, 11.96it/s]


In [8]:
df_censo

,ORDEN,NOMBRE REGIÓN,CÓDIGO REGIÓN,NOMBRE PROVINCIA,CÓDIGO PROVINCIA,NOMBRE COMUNA,CÓDIGO COMUNA,EDAD,TOTAL POBLACIÓN EFECTIVAMENTE CENSADA,HOMBRES,MUJERES,TOTAL ÁREA URBANA,HOMBRES ÁREA URBANA,MUJERES ÁREA URBANA,TOTAL ÁREA RURAL,HOMBRES ÁREA RURAL,MUJERES ÁREA RURAL
1,ORDEN,NOMBRE REGION,CODIGO REGION,NOMBRE PROVINCIA,CODIGO PROVINCIA,NOMBRE COMUNA,CODIGO COMUNA,EDAD,TOTAL POBLACION EFECTIVAMENTE CENSADA,HOMBRES,MUJERES,TOTAL AREA URBANA,HOMBRES AREA URBANA,MUJERES AREA URBANA,TOTAL AREA RURAL,HOMBRES AREA RURAL,MUJERES AREA RURAL
2,0,PAIS,País,PAIS,País,PAIS,País,Total País,17574003,8601989,8972014,15424263,7466848,7957415,2149740,1135141,1014599
3,1,PAIS,País,PAIS,País,PAIS,País,0,214682,109390,105292,191763,97904,93859,22919,11486,11433
4,2,PAIS,País,PAIS,País,PAIS,País,1,236356,120289,116067,210190,106888,103302,26166,13401,12765
5,3,PAIS,País,PAIS,País,PAIS,País,2,240792,122792,118000,213845,108953,104892,26947,13839,13108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35391,35389,MAGALLANES Y DE LA ANTARTICA CHILENA,12,ULTIMA ESPERANZA,124,TORRES DEL PAINE,12402,96,0,0,0,0,0,0,0,0,0
35392,35390,MAGALLANES Y DE LA ANTARTICA CHILENA,12,ULTIMA ESPERANZA,124,TORRES DEL PAINE,12402,97,0,0,0,0,0,0,0,0,0
35393,35391,MAGALLANES Y DE LA ANTARTICA CHILENA,12,ULTIMA ESPERANZA,124,TORRES DEL PAINE,12402,98,0,0,0,0,0,0,0,0,0
35394,35392,MAGALLANES Y DE LA ANTARTICA CHILENA,12,ULTIMA ESPERANZA,124,TORRES DEL PAINE,12402,99,0,0,0,0,0,0,0,0,0


In [9]:
seleccion_censo_comunas = df_censo.loc[df_censo["EDAD"].str.contains("Total")]

In [10]:
seleccion_censo_comunas

,ORDEN,NOMBRE REGIÓN,CÓDIGO REGIÓN,NOMBRE PROVINCIA,CÓDIGO PROVINCIA,NOMBRE COMUNA,CÓDIGO COMUNA,EDAD,TOTAL POBLACIÓN EFECTIVAMENTE CENSADA,HOMBRES,MUJERES,TOTAL ÁREA URBANA,HOMBRES ÁREA URBANA,MUJERES ÁREA URBANA,TOTAL ÁREA RURAL,HOMBRES ÁREA RURAL,MUJERES ÁREA RURAL
2,0,PAIS,País,PAIS,País,PAIS,País,Total País,17574003,8601989,8972014,15424263,7466848,7957415,2149740,1135141,1014599
104,102,ARICA Y PARINACOTA,15,ARICA,151,ARICA,15101,Total Comuna,221364,109389,111975,205079,99711,105368,16285,9678,6607
206,204,ARICA Y PARINACOTA,15,ARICA,151,CAMARONES,15102,Total Comuna,1255,726,529,0,0,0,1255,726,529
308,306,ARICA Y PARINACOTA,15,PARINACOTA,152,PUTRE,15201,Total Comuna,2765,2054,711,2152,1693,459,613,361,252
410,408,ARICA Y PARINACOTA,15,PARINACOTA,152,GENERAL LAGOS,15202,Total Comuna,684,412,272,0,0,0,684,412,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34886,34884,MAGALLANES Y DE LA ANTARTICA CHILENA,12,TIERRA DEL FUEGO,123,PORVENIR,12301,Total Comuna,6801,3993,2808,6062,3383,2679,739,610,129
34988,34986,MAGALLANES Y DE LA ANTARTICA CHILENA,12,TIERRA DEL FUEGO,123,PRIMAVERA,12302,Total Comuna,1158,930,228,0,0,0,1158,930,228
35090,35088,MAGALLANES Y DE LA ANTARTICA CHILENA,12,TIERRA DEL FUEGO,123,TIMAUKEL,12303,Total Comuna,405,343,62,0,0,0,405,343,62
35192,35190,MAGALLANES Y DE LA ANTARTICA CHILENA,12,ULTIMA ESPERANZA,124,NATALES,12401,Total Comuna,21477,10992,10485,19180,9515,9665,2297,1477,820


In [11]:
seleccion_censo_comunas.drop(2, axis='rows', inplace=True)

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
seleccion_censo_comunas

,ORDEN,NOMBRE REGIÓN,CÓDIGO REGIÓN,NOMBRE PROVINCIA,CÓDIGO PROVINCIA,NOMBRE COMUNA,CÓDIGO COMUNA,EDAD,TOTAL POBLACIÓN EFECTIVAMENTE CENSADA,HOMBRES,MUJERES,TOTAL ÁREA URBANA,HOMBRES ÁREA URBANA,MUJERES ÁREA URBANA,TOTAL ÁREA RURAL,HOMBRES ÁREA RURAL,MUJERES ÁREA RURAL
104,102,ARICA Y PARINACOTA,15,ARICA,151,ARICA,15101,Total Comuna,221364,109389,111975,205079,99711,105368,16285,9678,6607
206,204,ARICA Y PARINACOTA,15,ARICA,151,CAMARONES,15102,Total Comuna,1255,726,529,0,0,0,1255,726,529
308,306,ARICA Y PARINACOTA,15,PARINACOTA,152,PUTRE,15201,Total Comuna,2765,2054,711,2152,1693,459,613,361,252
410,408,ARICA Y PARINACOTA,15,PARINACOTA,152,GENERAL LAGOS,15202,Total Comuna,684,412,272,0,0,0,684,412,272
512,510,TARAPACA,1,IQUIQUE,11,IQUIQUE,1101,Total Comuna,191468,94897,96571,189065,93480,95585,2403,1417,986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34886,34884,MAGALLANES Y DE LA ANTARTICA CHILENA,12,TIERRA DEL FUEGO,123,PORVENIR,12301,Total Comuna,6801,3993,2808,6062,3383,2679,739,610,129
34988,34986,MAGALLANES Y DE LA ANTARTICA CHILENA,12,TIERRA DEL FUEGO,123,PRIMAVERA,12302,Total Comuna,1158,930,228,0,0,0,1158,930,228
35090,35088,MAGALLANES Y DE LA ANTARTICA CHILENA,12,TIERRA DEL FUEGO,123,TIMAUKEL,12303,Total Comuna,405,343,62,0,0,0,405,343,62
35192,35190,MAGALLANES Y DE LA ANTARTICA CHILENA,12,ULTIMA ESPERANZA,124,NATALES,12401,Total Comuna,21477,10992,10485,19180,9515,9665,2297,1477,820


## Verifico nombres de comunas de ambos dataset

In [13]:
comunas = df_comunas['Nombre Comuna'].unique()    

In [17]:
for comuna_censo in seleccion_censo_comunas['NOMBRE COMUNA'].unique():
    if not comuna_censo in comunas:
        print(comuna_censo)

In [15]:
seleccion_censo_comunas.loc[seleccion_censo_comunas['NOMBRE COMUNA'] == 'COYHAIQUE', 'NOMBRE COMUNA'] = 'COIHAIQUE'
seleccion_censo_comunas.loc[seleccion_censo_comunas['NOMBRE COMUNA'] == 'PAIGUANO', 'NOMBRE COMUNA'] = 'PAIHUANO'
seleccion_censo_comunas.loc[seleccion_censo_comunas['NOMBRE COMUNA'] == 'TILTIL', 'NOMBRE COMUNA'] = 'TIL TIL'
seleccion_censo_comunas.loc[seleccion_censo_comunas['NOMBRE COMUNA'] == 'AYSEN', 'NOMBRE COMUNA'] = 'AISEN'

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [16]:
# Reset el index para realizar feather

seleccion_censo_comunas.reset_index(inplace = True)

path_processed = "../data/processed/censo"
os.makedirs(path_processed, exist_ok = True) 

# Guardamos dataset como archivo feather
seleccion_censo_comunas.to_feather(f'{path_processed}/Censo2017_feather')